In [1]:
import pandas as pd
import xlsxwriter as writer

In [2]:
def load_selected_features(filepath):
    return pd.read_csv(filepath)

# Function to rank features based on their importance and calculate average rank
def rank_features(svm_features, xgboost_features, top_n=20):
    # Rank features separately for SVM and XGBoost
    svm_features['SVM_Rank'] = svm_features['Importance'].rank(ascending=False, method='min')
    xgboost_features['XGBoost_Rank'] = xgboost_features['Importance'].rank(ascending=False, method='min')
    
    # Combine the features and calculate the average rank
    combined_features = pd.merge(svm_features, xgboost_features, on='Feature', how='outer', suffixes=('_SVM', '_XGBoost'))
    combined_features['Average_Rank'] = combined_features[['SVM_Rank', 'XGBoost_Rank']].mean(axis=1)
    
    # Select top features based on average rank
    top_features = combined_features.nsmallest(top_n, 'Average_Rank')
    
    # Combine classifier names
    def combine_classifiers(row):
        classifiers = []
        if not pd.isna(row['Classifier_SVM']):
            classifiers.append('SVM')
        if not pd.isna(row['Classifier_XGBoost']):
            classifiers.append('XGBoost')
        return ', '.join(classifiers)
    
    top_features['Classifier'] = top_features.apply(combine_classifiers, axis=1)
    return top_features[['Feature', 'Average_Rank', 'Classifier']]

In [3]:
# Load the selected features for each comparison
ctl_b_features = load_selected_features('../GSEA/miRNA/50_ctl_b.csv')
ctl_c_features = load_selected_features('../GSEA/miRNA/50_ctl_c.csv')
b_c_features = load_selected_features('../GSEA/miRNA/50_b_c.csv')

# Separate the features by classifier
ctl_b_svm_features = ctl_b_features[ctl_b_features['Classifier'] == 'SVM']
ctl_b_xgboost_features = ctl_b_features[ctl_b_features['Classifier'] == 'XGBoost']
ctl_c_svm_features = ctl_c_features[ctl_c_features['Classifier'] == 'SVM']
ctl_c_xgboost_features = ctl_c_features[ctl_c_features['Classifier'] == 'XGBoost']
b_c_svm_features = b_c_features[b_c_features['Classifier'] == 'SVM']
b_c_xgboost_features = b_c_features[b_c_features['Classifier'] == 'XGBoost']

# Rank and get top 20 features for each comparison
top_ctl_b_features = rank_features(ctl_b_svm_features, ctl_b_xgboost_features)
top_ctl_c_features = rank_features(ctl_c_svm_features, ctl_c_xgboost_features)
top_b_c_features = rank_features(b_c_svm_features, b_c_xgboost_features)

/var/folders/c_/rs13jmts3nbf9xfm917mh9p80000gn/T/ipykernel_6747/2661197327.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  svm_features['SVM_Rank'] = svm_features['Importance'].rank(ascending=False, method='min')
/var/folders/c_/rs13jmts3nbf9xfm917mh9p80000gn/T/ipykernel_6747/2661197327.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xgboost_features['XGBoost_Rank'] = xgboost_features['Importance'].rank(ascending=False, method='min')
/var/folders/c_/rs13jmts3nbf9xfm917mh9p80000gn/T/ipykernel_6747/26

In [4]:
# Create the final table
final_table = pd.DataFrame()

# Add top features and classifiers for ctl_b
final_table['CTL_B'] = top_ctl_b_features['Feature'].values
final_table['Classifier'] = top_ctl_b_features['Classifier'].values

# Add top features and classifiers for ctl_c
final_table['CTL_C'] = top_ctl_c_features['Feature'].values
final_table['Classifier1'] = top_ctl_c_features['Classifier'].values
# rename Classifier1 to Classifier
final_table = final_table.rename(columns={'Classifier1': 'Classifier'})

# Add top features and classifiers for b_c
final_table['B_C'] = top_b_c_features['Feature'].values
final_table['Classifier2'] = top_b_c_features['Classifier'].values
# rename Classifier2 to Classifier
final_table = final_table.rename(columns={'Classifier2': 'Classifier'})

# Save the final table to a CSV file
# final_table.to_excel('./output/top_20_miRNA_comparisons.xlsx', index=False)

In [5]:
import pandas as pd

# Load the final table from Excel
final_table = pd.read_excel('./output/top_20_miRNA_comparisons.xlsx')

# Flatten all miRNA values into a single list and count occurrences
miRNAs = final_table['CTL_B'].tolist() + final_table['CTL_C'].tolist() + final_table['B_C'].tolist()
miRNA_counts = pd.Series(miRNAs).value_counts()

# Function to determine the color based on the columns each miRNA appears in
def determine_color(miRNA):
    in_ctl_b = miRNA in final_table['CTL_B'].values
    in_ctl_c = miRNA in final_table['CTL_C'].values
    in_b_c = miRNA in final_table['B_C'].values

    if in_ctl_b and in_ctl_c and in_b_c:
        return 'Red'
    elif in_ctl_b and in_ctl_c:
        return 'Blue'
    elif in_ctl_b and in_b_c:
        return 'Green'
    elif in_ctl_c and in_b_c:
        return 'Magenta'
    else:
        return 'No Color'

# Create a dictionary to store the miRNA and their associated colors
miRNA_color_dict = {miRNA: determine_color(miRNA) for miRNA in miRNA_counts.index}

# Print each unique miRNA and its associated color
for miRNA, color in miRNA_color_dict.items():
    print(f"miRNA: {miRNA}, Color: {color}")

# Save the miRNA color dictionary to a new Excel file for reference
# miRNA_color_df = pd.DataFrame(list(miRNA_color_dict.items()), columns=['miRNA', 'Color'])
# miRNA_color_df.to_excel('./output/miRNA_color_mapping.xlsx', index=False)

miRNA: hsa-miR-1469, Color: Blue
miRNA: hsa-miR-5100, Color: Magenta
miRNA: hsa-miR-7110-5p, Color: Green
miRNA: hsa-miR-6802-5p, Color: Blue
miRNA: hsa-miR-3184-5p, Color: No Color
miRNA: hsa-miR-6729-5p, Color: No Color
miRNA: hsa-miR-1343-3p, Color: No Color
miRNA: hsa-miR-1307-3p, Color: No Color
miRNA: hsa-miR-4532, Color: No Color
miRNA: hsa-miR-4783-3p, Color: No Color
miRNA: hsa-miR-1290, Color: No Color
miRNA: hsa-miR-6076, Color: No Color
miRNA: hsa-miR-4685-3p, Color: No Color
miRNA: hsa-miR-6780a-5p, Color: No Color
miRNA: hsa-miR-4440, Color: No Color
miRNA: hsa-miR-6793-5p, Color: No Color
miRNA: hsa-miR-4754, Color: No Color
miRNA: hsa-miR-1260b, Color: No Color
miRNA: hsa-miR-320d, Color: No Color
miRNA: hsa-miR-504-3p, Color: No Color
miRNA: hsa-miR-1185-2-3p, Color: No Color
miRNA: hsa-miR-4472, Color: No Color
miRNA: hsa-miR-495-5p, Color: No Color
miRNA: hsa-miR-17-3p, Color: No Color
miRNA: hsa-miR-3652, Color: No Color
miRNA: hsa-miR-5698, Color: No Color
miRNA: h